# Cleaning and EDA (R)

In [9]:
!pip install rpy2
%load_ext rpy2.ipython

In [10]:
%%R
# load packages
from google.colab import files
uploaded = files.upload()

library(tidyverse)
library(dplyr)

#load data
retail <- read_csv("online_retail_data.csv")

# clean and prep data
###############################################################################
# 1. CLEAN RAW DATA
###############################################################################

# Fix column name
retail <- retail %>%
  rename(CustomerID = `Customer ID`)

# Remove rows where CustomerID is missing
retail <- retail %>%
  filter(!is.na(CustomerID))

# Convert variables
retail <- retail %>%
  mutate(
    InvoiceDate = as.POSIXct(InvoiceDate, format = "%m/%d/%Y %H:%M"),
    CustomerID  = as.factor(CustomerID),
    Country     = as.factor(Country)
  )

# Define analysis date for recency
analysis_date <- max(retail$InvoiceDate)



###############################################################################
# 2. SEPARATE PURCHASES AND RETURNS
###############################################################################

# eliminate impossible quantities
retail <- retail %>%
  filter(Quantity < 500)

# Purchase invoices = Quantity > 0
purchase_data <- retail %>%
  filter(Quantity > 0)

# Returns = Quantity < 0
return_data <- retail %>%
  filter(Quantity < 0)


###############################################################################
# 3. CUSTOMER-LEVEL FEATURE CREATION (PURCHASE-ONLY)
###############################################################################

customer_data <- purchase_data %>%
  group_by(CustomerID) %>%
  summarise(
    repeat_purchase = if_else(n_distinct(Invoice) > 1, 1, 0),
    n_purchases = n_distinct(Invoice),
    total_spent = sum(Quantity * Price, na.rm = TRUE),
    avg_spent_per_purchase = total_spent / n_purchases,
    first_purchase = min(InvoiceDate),
    last_purchase  = max(InvoiceDate),
    recency_days = as.numeric(analysis_date - max(InvoiceDate), units = "days"),
    customer_lifetime_days =
      as.numeric(max(InvoiceDate) - min(InvoiceDate), units = "days"),
    avg_qty = mean(Quantity, na.rm = TRUE),
    Country = first(Country)
  ) %>%
  ungroup()




###############################################################################
# 4. ADD RETURN BEHAVIOR (FROM return_data)
###############################################################################

# Did the customer EVER return something?
return_flags <- return_data %>%
  group_by(CustomerID) %>%
  summarise(any_return = 1)

# Join return indicator (customers with no returns get NA → replace with 0)
customer_data <- customer_data %>%
  left_join(return_flags, by = "CustomerID") %>%
  mutate(any_return = if_else(is.na(any_return), 0, any_return))



#################################################################################
# EDA
###########################################################################

# Check summary statistics
customer_data%>% select(repeat_purchase, n_purchases,total_spent, recency_days,customer_lifetime_days) %>% summary()

# Print the Proportion of Repeat Purchases
repeat_counts<- table(customer_data$repeat_purchase)
repeat_prop <- prop.table(repeat_counts)
print(repeat_counts)
cat(sprintf("Proportion of Repeat Purchasers",repeat_prop["1"]*100))

# Distribution of Recency
p1 <-customer_data%>%
  ggplot(aes(x = recency_days, fill = factor(repeat_purchase)))+
  geom_histogram(binwidth= 25, position ="stack")+
  labs(
    title = "Distribution of Recency by Repeat Purchase Status",
    x = "Recency",
    y= "Number of Customers",
    fill ="Repeat Purchase (1=Yes)")

# Distribution of Total Spent
p2<-customer_data%>%
  ggplot(aes(x =total_spent,fill = factor(repeat_purchase))) +
  geom_histogram(binwidth = 450,position = "identity",alpha = 0.5) +
  labs(title = "Distribution of Total Spent (Zoomed)",
    x = "Total Spent",
    y = "Number of Customers",
    fill ="Repeat Purchase (1=Yes)")+xlim(0, 5000)

print(p1)
print(p2)


#Looking at mean values
customer_data %>%
  group_by(repeat_purchase) %>%
  summarise(Mean_Recency = mean(recency_days),
    Mean_Total_Spent = mean(total_spent),
    Mean_Invoices = mean(n_purchases),
    Proportion_Returned = mean(any_return))

# Correlation matrix
cor_matrix <- customer_data %>% select(repeat_purchase, n_purchases,total_spent,avg_spent_per_purchase,
         recency_days, customer_lifetime_days, avg_qty) %>% cor()
print(cor_matrix)

# Looking at distribution of Country variable
customer_data%>%
  count(Country,sort = TRUE)%>%
  top_n(10, n)

customer_data%>%
  group_by(Country) %>%
  filter(n() > 100) %>%
  summarise(Customer_Count = n(),
    Repeat_Rate = mean(repeat_purchase)
  ) %>% arrange(desc(Repeat_Rate))
###############################################################################
# 5. RESULT
###############################################################################

# customer_data now contains:
# CustomerID
# repeat_purchase
# n_invoices
# total_spent
# first_purchase
# last_purchase
# avg_spent_per_purchase
# recency_days
# customer_lifetime_days
# avg_qty
# Country
# any_return


RParsingError: Parsing status not OK - PARSING_STATUS.PARSE_ERROR

# Ridge Regression (Python)

1.   List item
2.   List item



In [ ]:
from google.colab import files
uploaded = files.upload()

Saving customer_data.csv to customer_data.csv


In [ ]:
import pandas as pd
data = pd.read_csv("customer_data.csv")

def map_region(country):
    if country in ["United Kingdom", "EIRE", "Channel Islands"]:
        return "UK_Ireland"
    elif country in ["France", "Germany", "Belgium",
                     "Netherlands", "Switzerland", "Austria"]:
        return "Western_Europe"
    elif country in ["Sweden", "Denmark", "Finland",
                     "Norway", "Iceland"]:
        return "Northern_Europe"
    elif country in ["Spain", "Italy", "Portugal",
                     "Greece", "Malta"]:
        return "Southern_Europe"
    elif country in ["Poland", "Czech Republic", "Lithuania"]:
        return "Eastern_Europe"
    elif country in ["Israel", "Saudi Arabia",
                     "United Arab Emirates", "Lebanon", "Bahrain"]:
        return "Middle_East"
    elif country in ["Japan", "Singapore", "Australia"]:
        return "Asia_Pacific"
    elif country in ["USA", "Canada", "Brazil"]:
        return "Americas"
    elif country == "RSA":
        return "Africa"
    else:
        return "Other"

data["Region"] = data["Country"].map(map_region)

feature_cols = [
    "avg_spent_per_purchase",
    "avg_qty",
    "any_return",
    "Region"
]

X = data[feature_cols]
y = data["repeat_purchase"].astype(int)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=123,
    stratify=y
)

print(y_train.mean())

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_features = [
    "avg_spent_per_purchase",
    "avg_qty",
    "any_return"
]

categorical_features = ["Region"]

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
)

from sklearn.linear_model import LogisticRegressionCV
import numpy as np

Cs = np.logspace(-3, 3, 15)

ridge_logit = LogisticRegressionCV(
    Cs=Cs,
    cv=5,
    penalty="l2",
    solver="lbfgs",
    scoring="neg_log_loss",
    max_iter=1000,
    n_jobs=-1
)

from sklearn.pipeline import Pipeline

model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("classifier", ridge_logit)
])

model.fit(X_train, y_train)

from sklearn.metrics import roc_auc_score, log_loss, classification_report

y_prob = model.predict_proba(X_test)[:, 1]
y_pred = (y_prob > 0.5).astype(int)

print("AUC:", roc_auc_score(y_test, y_prob))
print("Log loss:", log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))

best_C = model.named_steps["classifier"].C_
best_lambda = 1 / best_C

print("Best C:", best_C)
print("Lambda:", best_lambda)

import numpy as np

base_prob = np.mean(y_train)
base_auc = roc_auc_score(y_test, np.full_like(y_test, base_prob, dtype=float))
base_logloss = log_loss(y_test, np.full_like(y_test, base_prob, dtype=float))

print("Baseline AUC:", base_auc)
print("Baseline logloss:", base_logloss)

0.6542272126816381
AUC: 0.6525874800958025
Log loss: 0.5885052175167126
              precision    recall  f1-score   support

           0       0.57      0.04      0.07       449
           1       0.66      0.98      0.79       849

    accuracy                           0.66      1298
   macro avg       0.61      0.51      0.43      1298
weighted avg       0.63      0.66      0.54      1298

Best C: [0.05179475]
Lambda: [19.30697729]
Baseline AUC: 0.5
Baseline logloss: 0.6448823115311909


In [ ]:
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    precision_recall_curve,
    accuracy_score
)

# 1. Predicted probabilities for class = 1
y_prob = model.predict_proba(X_test)[:, 1]

# 2. Choose threshold (change this to see trade-offs)
threshold = 0.5
y_pred = (y_prob >= threshold).astype(int)

# 3. Confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# 4. Metrics
sensitivity = tp / (tp + fn)        # Recall for class 1
specificity = tn / (tn + fp)        # Recall for class 0
precision   = tp / (tp + fp)        # Positive predictive value
accuracy    = (tp + tn) / (tp + tn + fp + fn)

print(f"Sensitivity (Recall 1): {sensitivity:.3f}")
print(f"Specificity (Recall 0): {specificity:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Accuracy: {accuracy:.3f}")

# 5. ROC-derived quantities (threshold-free)
fpr, tpr, roc_thresholds = roc_curve(y_test, y_prob)
specificity_curve = 1 - fpr

# 6. Precision-Recall curve
pr_precision, pr_recall, pr_thresholds = precision_recall_curve(y_test, y_prob)

# 7. Example threshold selection rules

# Youden’s J statistic
j_stat = tpr - fpr
best_threshold_j = roc_thresholds[j_stat.argmax()]

# Fixed minimum specificity (e.g., >= 0.6)
min_spec = 0.6
idx = np.where(specificity_curve >= min_spec)[0]
threshold_min_spec = roc_thresholds[idx[0]] if len(idx) > 0 else None

# Top X% highest-risk customers (e.g., top 20%)
top_pct = 20
threshold_top_pct = np.percentile(y_prob, 100 - top_pct)

print("\nExample thresholds:")
print(f"Best threshold (Youden's J): {best_threshold_j:.3f}")
print(f"Threshold w/ specificity ≥ {min_spec}: {threshold_min_spec}")
print(f"Threshold for top {top_pct}% risk: {threshold_top_pct:.3f}")


Sensitivity (Recall 1): 0.985
Specificity (Recall 0): 0.038
Precision: 0.659
Accuracy: 0.657

Example thresholds:
Best threshold (Youden's J): 0.812
Threshold w/ specificity ≥ 0.6: inf
Threshold for top 20% risk: 0.862


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving customer_first_purchase.csv to customer_first_purchase.csv


In [ ]:

# ============================================================
# Ridge Logistic Regression on FIRST-PURCHASE customer data
# with logically selected features
# ============================================================

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score, log_loss, classification_report

# ------------------------------------------------------------
# 1. Load data
# ------------------------------------------------------------

data = pd.read_csv("customer_first_purchase.csv")

# Ensure target and basic types
data["repeat_purchase"] = data["repeat_purchase"].astype(int)
data["Country"] = data["Country"].astype(str)

# ------------------------------------------------------------
# 2. Create REGION from Country (more stable than raw Country)
# ------------------------------------------------------------

def map_region(country: str) -> str:
    c = country.strip()
    if c in ["United Kingdom", "EIRE", "Channel Islands"]:
        return "UK_Ireland"
    elif c in ["France", "Germany", "Belgium", "Netherlands", "Switzerland", "Austria"]:
        return "Western_Europe"
    elif c in ["Sweden", "Denmark", "Finland", "Norway", "Iceland"]:
        return "Northern_Europe"
    elif c in ["Spain", "Italy", "Portugal", "Greece", "Malta"]:
        return "Southern_Europe"
    elif c in ["Poland", "Czech Republic", "Lithuania"]:
        return "Eastern_Europe"
    elif c in ["Israel", "Saudi Arabia", "United Arab Emirates", "Lebanon", "Bahrain"]:
        return "Middle_East"
    elif c in ["Japan", "Singapore", "Australia"]:
        return "Asia_Pacific"
    elif c in ["USA", "Canada", "Brazil"]:
        return "Americas"
    elif c in ["RSA"]:
        return "Africa"
    else:
        return "Other"

data["Region"] = data["Country"].apply(map_region)

# Treat purchase_dow as categorical (weekday effect is not linear)
data["purchase_dow"] = data["purchase_dow"].astype("category")

# ------------------------------------------------------------
# 3. Logically chosen features
# ------------------------------------------------------------
# - basket_spend: overall monetary value of first basket
# - basket_qty: total units in basket
# - basket_lines: number of distinct line items
# - purchase_hour: time-of-day pattern
# - purchase_dow: weekday/weekend + weekday effects
# - Region: broad geography instead of high-card Country

feature_cols = [
    "basket_spend",
    "basket_qty",
    "basket_lines",
    "purchase_hour",
    "purchase_dow",
    "Region"
]

X = data[feature_cols]
y = data["repeat_purchase"]

# ------------------------------------------------------------
# 4. Train/test split (stratified)
# ------------------------------------------------------------

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.30,
    random_state=123,
    stratify=y
)

# ------------------------------------------------------------
# 5. Preprocessing
# ------------------------------------------------------------

numeric_features = [
    "basket_spend",
    "basket_qty",
    "basket_lines",
    "purchase_hour"
]

categorical_features = ["purchase_dow", "Region"]

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
)

# ------------------------------------------------------------
# 6. Ridge logistic regression (L2 penalty + CV on C)
# ------------------------------------------------------------

Cs = np.logspace(-3, 3, 15)   # C = 1 / lambda

ridge_logit = LogisticRegressionCV(
    Cs=Cs,
    cv=5,
    penalty="l2",
    solver="lbfgs",
    scoring="neg_log_loss",
    max_iter=2000,
    n_jobs=-1
)

model = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("classifier", ridge_logit)
    ]
)

# ------------------------------------------------------------
# 7. Fit model
# ------------------------------------------------------------

model.fit(X_train, y_train)

# ------------------------------------------------------------
# 8. Evaluate on test set
# ------------------------------------------------------------

y_prob = model.predict_proba(X_test)[:, 1]
y_pred = (y_prob > 0.5).astype(int)

auc = roc_auc_score(y_test, y_prob)
ll  = log_loss(y_test, y_prob)

print("Ridge Logistic Regression (First Purchase, Logical Features)")
print("-----------------------------------------------------------")
print("AUC:", auc)
print("Log loss:", ll)
print()
print(classification_report(y_test, y_pred))

# ------------------------------------------------------------
# 9. Baseline comparison
# ------------------------------------------------------------

base_prob = y_train.mean()

baseline_auc = roc_auc_score(
    y_test,
    np.full_like(y_test, base_prob, dtype=float)
)

baseline_logloss = log_loss(
    y_test,
    np.full_like(y_test, base_prob, dtype=float)
)

print("Baseline Results")
print("----------------")
print("Baseline AUC:", baseline_auc)
print("Baseline Log loss:", baseline_logloss)

# ------------------------------------------------------------
# 10. Regularization strength chosen by CV
# ------------------------------------------------------------

best_C = model.named_steps["classifier"].C_[0]
best_lambda = 1 / best_C

print()
print("Chosen C (1/lambda):", best_C)
print("Equivalent lambda:", best_lambda)


Ridge Logistic Regression (First Purchase, Logical Features)
-----------------------------------------------------------
AUC: 0.5366630771160923
Log loss: 0.642833853485938

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       448
           1       0.66      1.00      0.79       854

    accuracy                           0.66      1302
   macro avg       0.33      0.50      0.40      1302
weighted avg       0.43      0.66      0.52      1302

Baseline Results
----------------
Baseline AUC: 0.5
Baseline Log loss: 0.6437088287456223

Chosen C (1/lambda): 0.001
Equivalent lambda: 1000.0


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Random Forest (R)

In [ ]:
!pip install rpy2
%load_ext rpy2.ipython

In [ ]:
%%R
#Set up =============================================

#Load in packages
library(dplyr) #Data manipulation
library(tidyr) #Data manipulation
install.packages("tree", repos="http://cran.us.r-project.org")
install.packages("randomForest", repos="http://cran.us.r-project.org")
library(tree) #tree()
library(randomForest) #randomForest()

#First Purchase Data ================================

firstPurchaseDf <- read.csv('customer_first_purchase.csv')

#Factor categorical columns and recategorize countries
#tree() function cannot handle more than approx. 30 levels and there are over that many countries listed
firstPurchaseDf <- firstPurchaseDf |> mutate(Repeats = factor(ifelse(repeat_purchase == 1, 'Yes', 'No')),
            Region = case_when(Country %in% c("United Kingdom", "EIRE", "Channel Islands") ~ "UK_Ireland",
                            Country %in% c("France", "Germany", "Belgium","Netherlands", "Switzerland", "Austria") ~ 'Western_Europe',
                            Country %in% c("Sweden", "Denmark", "Finland", "Norway", "Iceland") ~ 'Northern_Europe',
                            Country %in% c("Spain", "Italy", "Portugal", "Greece", "Malta") ~ 'Southern_Europe',
                            Country %in% c("Poland", "Czech Republic", "Lithuania") ~ 'Eastern_Europe',
                            Country %in% c("Israel", "Saudi Arabia", "United Arab Emirates", "Lebanon", "Bahrain") ~ 'Middle_East',
                            Country %in% c("Japan", "Singapore", "Australia") ~ 'Asia_Pacific',
                            Country %in% c("USA", "Canada", "Brazil") ~ 'Americas',
                            Country == "RSA" ~ 'Africa'),
            Region = factor(replace_na(Region, 'Other')))

colnames(firstPurchaseDf)
#remove CustomerID, repeat_purchases, and Country columns
firstPurchaseDf <- firstPurchaseDf[,-c(1,9,10)]
colnames(firstPurchaseDf)

#Classification Decision Trees =============================================
firstDecTree <- tree(Repeats ~ ., firstPurchaseDf)
#summary(firstDecTree)

#Calculate accuracy
set.seed(1)
firstTrainSample <- sample(1:nrow(firstPurchaseDf), nrow(firstPurchaseDf)/2)
firstTestDf <- firstPurchaseDf[-firstTrainSample,] #training data
firstRepeatTest <- firstTestDf$Repeats #test data
tree.first <- tree(Repeats ~ ., firstPurchaseDf, subset = firstTrainSample) #decision tree
first.tree.pred <- predict(tree.first, firstTestDf, type = 'class')
table(first.tree.pred, firstRepeatTest) #confusion matrix

accuracy <- (0+1420)/(0+1420+750+0) #0.6543779
sensitivity <- (1420)/(1420+0) #1 (TP/(TP+FN))
specificity <- (0)/(0+750) #0 (TN/(TN+FP))

#Cannot prune single node decision tree

#Random Forest =============================================
set.seed(1)
firstRFTree <- randomForest(Repeats ~.,
    data = firstPurchaseDf,
    subset = firstTrainSample,
    mtry = 6,
    ntree = 1000,
    importance = TRUE)
firstRFTree

accuracy = (124+1234)/(124+1234+191+620) #0.626095
sensitivity <- (1234)/(1234+620) #0.6655879
specificity <- (124)/(124+191) #0.3936508



#===============================================#



#Customer Data ================================


#Load in Data
aggCustomerDf <- read.csv('customer_data.csv')

#Factor categorical columns and recategorize countries
#tree() function cannot handle more than approx. 30 levels and there are over that many countries listed
aggCustomerDf <- aggCustomerDf |> mutate(Repeats = factor(ifelse(repeat_purchase == 1, 'Yes', 'No')),
            Country = factor(Country),
            CustomerID = factor(CustomerID),
            Region = case_when(Country %in% c("United Kingdom", "EIRE", "Channel Islands") ~ "UK_Ireland",
                            Country %in% c("France", "Germany", "Belgium","Netherlands", "Switzerland", "Austria") ~ 'Western_Europe',
                            Country %in% c("Sweden", "Denmark", "Finland", "Norway", "Iceland") ~ 'Northern_Europe',
                            Country %in% c("Spain", "Italy", "Portugal", "Greece", "Malta") ~ 'Southern_Europe',
                            Country %in% c("Poland", "Czech Republic", "Lithuania") ~ 'Eastern_Europe',
                            Country %in% c("Israel", "Saudi Arabia", "United Arab Emirates", "Lebanon", "Bahrain") ~ 'Middle_East',
                            Country %in% c("Japan", "Singapore", "Australia") ~ 'Asia_Pacific',
                            Country %in% c("USA", "Canada", "Brazil") ~ 'Americas',
                            Country == "RSA" ~ 'Africa'),
            Region = factor(replace_na(Region, 'Other')),
            Returns = factor(ifelse(any_return == 1, 'Yes', 'No')))

colnames(aggCustomerDf)
#Remove multicollinear columns to Repeats: repeat_purchase, n_purchases, total_spent, customer_lifetime_days
#Since data has been grouped together by customer, whether a customer has bought from the company more than once
#is directly related to the number of purchases (n_purchase), the amount of money they have given the store (total_spent),
#and how long they have been a customer (customer_lifetime_days).
#Remove columns with too many levels for tree() functions to handle: CustomerID, Country
aggCustomerDf <- aggCustomerDf[,-c(1,2,3,4,9,11,12)]
colnames(aggCustomerDf)

#Classification Decision Trees =============================================
aggDecTree <- tree(Repeats ~ ., aggCustomerDf)
summary(aggDecTree)

#Calculate accuracy
set.seed(1)
aggTrainSample <- sample(1:nrow(aggCustomerDf), nrow(aggCustomerDf)/2)
aggTestDf <- aggCustomerDf[-aggTrainSample,] #training data
aggRepeatsTest <- aggTestDf$Repeats #test data
tree.agg <- tree(Repeats ~ ., aggCustomerDf, subset = aggTrainSample) #decision tree
tree.agg.pred <- predict(tree.agg, aggTestDf, type = 'class')
table(tree.agg.pred, aggRepeatsTest) #confusion matrix

accuracy <- (277+1327)/(277+1327+462+97) #0.7415626
sensitivity <- (1327)/(1327+97) #0.931882
specificity <- (277)/(277+462) #0.3748309

#Prune the decision tree
cv.tree(tree.agg, FUN = prune.misclass) #2 is the best size
aggPrunedTree <- prune.misclass(tree.agg, best = 2)
aggPredPrune <- predict(aggPrunedTree, aggTestDf, type = 'class')
table(aggPredPrune, aggRepeatsTest)

accuracy <- (277+1327)/(277+1327+462+97) #0.7415626 (no change)
sensitivity <- (1327)/(1327+97) #0.931882
specificity <- (277)/(277+462) #0.3748309


#Random Forest =============================================
set.seed(1)
aggRFTree <- randomForest(Repeats ~.,
    data = aggCustomerDf,
    subset = aggTrainSample,
    mtry = 6,
    ntree = 1000,
    importance = TRUE)
aggRFTree

accuracy = (745+1340)/(745+1340+12+66) #0.963939
sensitivity <- (1340)/(1340+12) #0.9911243
specificity <- (745)/(745+66) #0.918619


#Compare Plots ===============================================

#Compare random forest plots between first purchase data and aggregated customer data
plot(firstRFTree)
plot(aggRFTree)

#Compare aggregate data decision trees before and after pruning
plot(aggDecTree)
text(aggDecTree, pos = 3)

plot(aggPrunedTree)
text(aggPrunedTree, pos = 3)

#Compare decision and random forest tree for aggregated data
plot(aggDecTree)
plot(aggRFTree)

#Compare random forest importance of variables
importance(firstRFTree)
importance(aggRFTree)

varImpPlot(firstRFTree)
varImpPlot(aggRFTree)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'http://cran.us.r-project.org/src/contrib/tree_1.0-45.tar.gz'
Content type 'application/x-gzip' length 49853 bytes (48 KB)
downloaded 48 KB


The downloaded source packages are in
	‘/tmp/RtmpbVNhk2/downloaded_packages’
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'http://cran.us.r-project.org/src/contrib/randomForest_4.7-1.2.tar.gz'
Content type 'application/x-gzip' length 81974 bytes (80 KB)
downloaded 80 KB


The downloaded source packages are in
	‘/tmp/RtmpbVNhk2/downloaded_packages’
Error in file(file, "rt") : cannot open the connection
In addition: Warning message:
In file(file, "rt") :
  cannot open file 'customer_first_purchase.csv': No such file or directory
Error in file(file, "rt") : cannot open the connection


RInterpreterError: Failed to parse and evaluate line '#Set up =============================================\n\n#Load in packages\nlibrary(dplyr) #Data manipulation\nlibrary(tidyr) #Data manipulation\ninstall.packages("tree", repos="http://cran.us.r-project.org")\ninstall.packages("randomForest", repos="http://cran.us.r-project.org")\nlibrary(tree) #tree()\nlibrary(randomForest) #randomForest()\n\n#First Purchase Data ================================\n\nfirstPurchaseDf <- read.csv(\'customer_first_purchase.csv\')\n\n#Factor categorical columns and recategorize countries\n#tree() function cannot handle more than approx. 30 levels and there are over that many countries listed\nfirstPurchaseDf <- firstPurchaseDf |> mutate(Repeats = factor(ifelse(repeat_purchase == 1, \'Yes\', \'No\')),\n            Region = case_when(Country %in% c("United Kingdom", "EIRE", "Channel Islands") ~ "UK_Ireland",\n                            Country %in% c("France", "Germany", "Belgium","Netherlands", "Switzerl

# Neural Network (Python)

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report

import torch
import torch.nn as nn
import torch.optim as optim

# 1. Load and clean data
df = pd.read_csv("customer_data.csv")

# Drop ID, timestamps, and leakage features
df = df.drop(columns=[
    "CustomerID",
    "first_purchase",
    "last_purchase",
    "n_purchases",
    "total_spent",
    "avg_spent_per_purchase",
    "customer_lifetime_days"
])

# Target
y = df["repeat_purchase"].astype(int).values

# Features
X = df.drop(columns=["repeat_purchase"])
X = pd.get_dummies(X, columns=["Country"], drop_first=True)

feature_names = X.columns.tolist()
X = X.values

# 2. Train / val / test split (60 / 20 / 20)
X_trval, X_te, y_trval, y_te = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_tr, X_val, y_tr, y_val = train_test_split(
    X_trval, y_trval, test_size=0.25, random_state=42, stratify=y_trval
)  # 0.25 of 0.8 = 0.2

# 3. Standardization (fit on train only)
sc = StandardScaler().fit(X_tr)
X_tr = sc.transform(X_tr)
X_val = sc.transform(X_val)
X_te  = sc.transform(X_te)

# 4. Convert to PyTorch tensors
Xtr = torch.tensor(X_tr, dtype=torch.float32)
Xval = torch.tensor(X_val, dtype=torch.float32)
Xte  = torch.tensor(X_te,  dtype=torch.float32)

ytr = torch.tensor(y_tr.reshape(-1, 1), dtype=torch.float32)
yval = torch.tensor(y_val.reshape(-1, 1), dtype=torch.float32)
yte  = torch.tensor(y_te.reshape(-1, 1), dtype=torch.float32)

# 5. FFNN model
class FFNN(nn.Module):
    def __init__(self, d, h=[64, 64], p=0.2):
        super().__init__()
        layers = []
        in_dim = d
        for u in h:
            layers += [
                nn.Linear(in_dim, u),
                nn.ReLU(),
                nn.BatchNorm1d(u),
                nn.Dropout(p)
            ]
            in_dim = u
        layers += [nn.Linear(in_dim, 1)]  # logits
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

d = Xtr.shape[1]
model = FFNN(d, h=[64, 64], p=0.2)

criterion = nn.BCEWithLogitsLoss()
opt = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-3)

# 6. Training with early stopping on validation AUC
def predict_proba(model, X_tensor):
    model.eval()
    with torch.no_grad():
        logits = model(X_tensor)
        probs = torch.sigmoid(logits).view(-1).cpu().numpy()
    return probs

best_state = None
best_auc = -1
patience = 10
wait = 0
max_epochs = 200

y_val_np = y_val  # for AUC
for ep in range(max_epochs):
    model.train()
    opt.zero_grad()

    logits = model(Xtr)
    loss = criterion(logits, ytr)
    loss.backward()
    opt.step()

    # Validation AUC
    val_probs = predict_proba(model, Xval)
    val_auc = roc_auc_score(y_val_np, val_probs)

    print(f"Epoch {ep+1:3d} | train loss {loss.item():.4f} | val AUC {val_auc:.4f}")

    if val_auc > best_auc:
        best_auc = val_auc
        best_state = model.state_dict()
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print(f"Early stopping at epoch {ep+1}")
            break

# 7. Load best model and evaluate on test set
model.load_state_dict(best_state)

test_probs = predict_proba(model, Xte)
test_auc = roc_auc_score(y_te, test_probs)
test_pred = (test_probs >= 0.5).astype(int)

acc = accuracy_score(y_te, test_pred)
print(f"\nTest AUC: {test_auc:.4f}")
print(f"Test accuracy: {acc:.4f}")
print("\nClassification report:")
print(classification_report(y_te, test_pred, digits=2))


KeyboardInterrupt: 